In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.measure import shannon_entropy
from skimage.feature.texture import graycomatrix, graycoprops


# === KLASÖR YOLLARI ===
RAW_FOLDER = "D:/ITU VCAMP/github/health-ai-project/data/train-inme-var"
MASK_FOLDER = "D:/ITU VCAMP/github/health-ai-project/data/train-overlay"
OUTPUT_CSV = "extracted_features_from_overlay.csv"

# === ÖZELLİK ÇIKARTMA FONKSİYONU ===
def extract_features(region):
    features = {}
    features["mean_intensity"] = np.mean(region)
    features["std_intensity"] = np.std(region)
    features["variance"] = np.var(region)
    features["entropy"] = shannon_entropy(region)

    # Kenar sayısı (Canny)
    edges = cv2.Canny(region, 50, 150)
    features["edge_count"] = np.sum(edges > 0)

    # GLCM doku özellikleri (sadece contrast ve homojenlik)
    try:
        glcm = graycomatrix(region, [1], [0], 256, symmetric=True, normed=True)
        features["glcm_contrast"] = graycoprops(glcm, 'contrast')[0, 0]
        features["glcm_homogeneity"] = graycoprops(glcm, 'homogeneity')[0, 0]

    except:
        features["glcm_contrast"] = 0
        features["glcm_homogeneity"] = 0

    return features

# === TÜM GÖRÜNTÜLERİ İŞLE ===
data = []
for filename in os.listdir(RAW_FOLDER):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    raw_path = os.path.join(RAW_FOLDER, filename)
    mask_path = os.path.join(MASK_FOLDER, filename)

    raw_img = cv2.imread(raw_path, cv2.IMREAD_GRAYSCALE)
    mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if raw_img is None or mask_img is None:
        print(f"Atlandı: {filename}")
        continue

    # Maskelenmiş inme bölgesi
    region = cv2.bitwise_and(raw_img, raw_img, mask=mask_img)

    # Özellikleri çıkart
    feats = extract_features(region)
    feats["filename"] = filename
    feats["masked_pixel_count"] = np.sum(mask_img > 0)

    data.append(feats)
    print(f"{filename} işlendi.")

# === CSV'YE KAYDET ===
df = pd.DataFrame(data)
df.to_csv(OUTPUT_CSV, index=False)
print("✅ Özellikler kaydedildi ->", OUTPUT_CSV)


10002.png işlendi.
10003.png işlendi.
10017.png işlendi.
10024.png işlendi.
10028.png işlendi.
10033.png işlendi.
10036.png işlendi.
10038.png işlendi.
10039.png işlendi.
10045.png işlendi.
10046.png işlendi.
10047.png işlendi.
10049.png işlendi.
10050.png işlendi.
10052.png işlendi.
10053.png işlendi.
10059.png işlendi.
10060.png işlendi.
10062.png işlendi.
10063.png işlendi.
10067.png işlendi.
10068.png işlendi.
10072.png işlendi.
10073.png işlendi.
10075.png işlendi.
10077.png işlendi.
10078.png işlendi.
10088.png işlendi.
10094.png işlendi.
10095.png işlendi.
10101.png işlendi.
10105.png işlendi.
10108.png işlendi.
10113.png işlendi.
10116.png işlendi.
10120.png işlendi.
10123.png işlendi.
10134.png işlendi.
10143.png işlendi.
10145.png işlendi.
10146.png işlendi.
10150.png işlendi.
10151.png işlendi.
10153.png işlendi.
10154.png işlendi.
10155.png işlendi.
10156.png işlendi.
10158.png işlendi.
10160.png işlendi.
10164.png işlendi.
10179.png işlendi.
10181.png işlendi.
10187.png iş

In [4]:
import pandas as pd
import json

# CSV dosyasını oku
csv_path = "extracted_features_from_overlay.csv"
df = pd.read_csv(csv_path)

# Hedef özellik sütunları
feature_cols = [
    "mean_intensity",
    "std_intensity",
    "variance",
    "entropy",
    "edge_count",
    "glcm_contrast",
    "glcm_homogeneity"
]

# İstatistiksel eşikleri çıkart
thresholds = {}
for col in feature_cols:
    thresholds[col] = {
        "min": float(df[col].min()),
        "max": float(df[col].max()),
        "mean": float(df[col].mean()),
        "std": float(df[col].std())
    }

# JSON olarak kaydet
with open("feature_reference.json", "w") as f:
    json.dump(thresholds, f, indent=4)

print("✅ Özellik eşikleri kaydedildi: feature_reference.json")


✅ Özellik eşikleri kaydedildi: feature_reference.json


features apply

In [ ]:
import os
import cv2
import numpy as np
import json
from skimage.measure import shannon_entropy

# === YOLLAR ===
INPUT_FOLDER = "D:/ITU VCAMP/github/health-ai-project/data/all-rotated"
OUTPUT_FOLDER = "enhanced"
REFERENCE_PATH = "feature_reference.json"

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === EŞİKLERİ YÜKLE ===
with open(REFERENCE_PATH, "r") as f:
    ref = json.load(f)

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# === BÖLGE ÖZELLİKLERİ ===
def get_features(region):
    entropy = shannon_entropy(region)
    edge = cv2.Canny(region, 50, 150)
    edge_count = np.sum(edge > 0)
    mean = np.mean(region)
    std = np.std(region)
    var = np.var(region)
    return {
        "mean_intensity": mean,
        "std_intensity": std,
        "variance": var,
        "entropy": entropy,
        "edge_count": edge_count
    }

# === GÜRÜLTÜ AZALTICI ===
def denoise(region):
    return cv2.bilateralFilter(region, d=9, sigmaColor=75, sigmaSpace=75)

# === SHARPEN ===
def sharpen(region):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(region, -1, kernel)

# === AKILLI İYİLEŞTİRME ===
def smart_enhance(region, feats, ref):
    enhanced = region.copy()

    if feats["variance"] > ref["variance"]["mean"] and feats["edge_count"] < ref["edge_count"]["mean"]:
        enhanced = denoise(enhanced)

    if feats["std_intensity"] < ref["std_intensity"]["mean"]:
        enhanced = clahe.apply(enhanced)

    if feats["entropy"] < ref["entropy"]["mean"]:
        enhanced = sharpen(enhanced)

    if feats["mean_intensity"] > ref["mean_intensity"]["mean"] + ref["mean_intensity"]["std"]:
        enhanced = 255 - enhanced

    return enhanced

# === GÖRSELLERİ DOLAŞ ===
for fname in os.listdir(INPUT_FOLDER):
    if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    img_path = os.path.join(INPUT_FOLDER, fname)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Bozuk atlandı: {fname}")
        continue

    output = img.copy()
    h, w = img.shape
    win_size = 3
    step = 2

    for y in range(0, h - win_size, step):
        for x in range(0, w - win_size, step):
            region = img[y:y+win_size, x:x+win_size]
            feats = get_features(region)
            enhanced = smart_enhance(region, feats, ref)
            output[y:y+win_size, x:x+win_size] = enhanced

    out_path = os.path.join(OUTPUT_FOLDER, fname)
    cv2.imwrite(out_path, output)
    print(f"✅ İşlendi: {fname}")

print("🎯 Tüm görüntüler iyileştirildi (kafatası maskesi olmadan, 3x3/2 sliding window ile).")


FileNotFoundError: [WinError 3] Sistem belirtilen yolu bulamıyor: 'test_raw'

In [ ]:
import os
import json
import cv2
import numpy as np
from skimage.measure import shannon_entropy
from skimage.feature.texture import graycomatrix, graycoprops

#####################################
# 1. Ayarlar
#####################################
RAW_FOLDER = "data/all-rotated"         # Sadece ham CT görüntülerin olduğu klasör
OUTPUT_FOLDER = "test_enhanced" # İyileştirilmiş çıktıların klasörü
DEBUG_FOLDER = "debug_mask"     # İşlem yapılan bölgeleri görselleştirme
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(DEBUG_FOLDER, exist_ok=True)

FEATURE_JSON = "feature_reference.json"  # AŞAMA 1'de oluşturulmuş eşik verisi

WIN_SIZE = 48              # Sliding window boyutu
SIGMA_THRESHOLD = 0.5      # Eşik hassasiyeti (daha düşük = daha sıkı)
BRAIN_INTENSITY_THRESHOLD = 20  # Beyin içi kabul edilen minimum mean değer

#####################################
# 2. Eşik verilerini yükle
#####################################
with open(FEATURE_JSON, "r") as f:
    threshold_data = json.load(f)

#####################################
# 3. Fonksiyonlar
#####################################
def get_brain_mask(image):
    """Kafatası içini bulmak için basit eşik + morfoloji uygulaması."""
    _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), np.uint8)
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    return closed

def compute_features(patch):
    """Bir patch içindeki görsel özellikleri hesapla."""
    feats = {}
    feats["mean_intensity"] = np.mean(patch)
    feats["std_intensity"]  = np.std(patch)
    feats["variance"]       = np.var(patch)
    feats["entropy"]        = shannon_entropy(patch)

    edges = cv2.Canny(patch, 50, 150)
    feats["edge_count"]     = np.sum(edges > 0)

    try:
        glcm = graycomatrix(patch, [1], [0], levels=256, symmetric=True, normed=True)
        feats["glcm_contrast"] = graycoprops(glcm, 'contrast')[0, 0]
        feats["glcm_homogeneity"] = graycoprops(glcm, 'homogeneity')[0, 0]
    except:
        feats["glcm_contrast"] = 0
        feats["glcm_homogeneity"] = 0

    return feats

def is_similar_to_infarct(feats):
    """Patch, inme benzeri mi? Eşiklere göre karar verir."""
    match_score = 0
    for col, val in feats.items():
        if col not in threshold_data:
            continue

        mean_val = threshold_data[col]["mean"]
        std_val = threshold_data[col]["std"]
        lower = mean_val - SIGMA_THRESHOLD * std_val
        upper = mean_val + SIGMA_THRESHOLD * std_val

        if lower <= val <= upper:
            match_score += 1

    return match_score >= 4  # En az 4 özellik uyuşmalı

def enhance_patch(patch):
    """Kontrast + keskinlik artır."""
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    patch_clahe = clahe.apply(patch)

    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    patch_sharp = cv2.filter2D(patch_clahe, -1, sharpen_kernel)

    return patch_sharp

#####################################
# 4. Görüntüyü işle
#####################################
def process_image(raw_img):
    enhanced_img = raw_img.copy()
    brain_mask = get_brain_mask(raw_img)
    h, w = raw_img.shape

    # Debug mask: işlem uygulanan yerleri işaretlemek için
    debug_mask = np.zeros_like(raw_img, dtype=np.uint8)

    for row in range(0, h - WIN_SIZE, WIN_SIZE // 2):
        for col in range(0, w - WIN_SIZE, WIN_SIZE // 2):
            patch = raw_img[row:row+WIN_SIZE, col:col+WIN_SIZE]
            patch_mask = brain_mask[row:row+WIN_SIZE, col:col+WIN_SIZE]

            # Beyin dışında atla
            if np.mean(patch_mask) < 50 or np.mean(patch) < BRAIN_INTENSITY_THRESHOLD:
                continue

            feats = compute_features(patch)

            if is_similar_to_infarct(feats):
                enhanced = enhance_patch(patch)
                enhanced_img[row:row+WIN_SIZE, col:col+WIN_SIZE] = enhanced
                debug_mask[row:row+WIN_SIZE, col:col+WIN_SIZE] = 255

    return enhanced_img, debug_mask

#####################################
# 5. Tüm klasörü işle
#####################################
for filename in os.listdir(RAW_FOLDER):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    img_path = os.path.join(RAW_FOLDER, filename)
    raw = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if raw is None:
        print(f"Hatalı dosya atlandı: {filename}")
        continue

    enhanced_img, debug_mask = process_image(raw)

    cv2.imwrite(os.path.join(OUTPUT_FOLDER, filename), enhanced_img)
    cv2.imwrite(os.path.join(DEBUG_FOLDER, filename), debug_mask)

    print(f"Tamamlandı: {filename}")

print("✅ Tüm görüntüler işlendi ve kayıtlar yapıldı.")


Tamamlandı: 10000.png
Tamamlandı: 10001.png
Tamamlandı: 10002.png
Tamamlandı: 10003.png
Tamamlandı: 10004.png
Tamamlandı: 10005.png
Tamamlandı: 10006.png
Tamamlandı: 10008.png
Tamamlandı: 10009.png
Tamamlandı: 10010.png
Tamamlandı: 10011.png
Tamamlandı: 10012.png
Tamamlandı: 10013.png
Tamamlandı: 10014.png
Tamamlandı: 10015.png
Tamamlandı: 10016.png
Tamamlandı: 10017.png
Tamamlandı: 10018.png
Tamamlandı: 10019.png
Tamamlandı: 10021.png
Tamamlandı: 10022.png
Tamamlandı: 10023.png
Tamamlandı: 10024.png
Tamamlandı: 10025.png
Tamamlandı: 10026.png
Tamamlandı: 10027.png
Tamamlandı: 10028.png
Tamamlandı: 10030.png
Tamamlandı: 10032.png
Tamamlandı: 10033.png
Tamamlandı: 10034.png
Tamamlandı: 10035.png
Tamamlandı: 10036.png
Tamamlandı: 10037.png
Tamamlandı: 10038.png
Tamamlandı: 10039.png
Tamamlandı: 10040.png
Tamamlandı: 10041.png
Tamamlandı: 10042.png
Tamamlandı: 10043.png
Tamamlandı: 10044.png
Tamamlandı: 10045.png
Tamamlandı: 10046.png
Tamamlandı: 10047.png
Tamamlandı: 10048.png
Tamamlandı

KNN benzerlik vs...

In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Stroke CSV verisi
df = pd.read_csv("extracted_features_from_overlay.csv")

# Feature kolonları
feature_cols = ['mean_intensity','std_intensity','variance','entropy',
                'edge_count','glcm_contrast','glcm_homogeneity']

X = df[feature_cols].values

# Normalizasyon
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# KNN modeli (benzerlik için en yakın 1 komşu yeterli)
knn = NearestNeighbors(n_neighbors=1, metric='euclidean')
knn.fit(X_scaled)

print("✅ KNN modeli eğitildi.")


✅ KNN modeli eğitildi.


In [15]:
import cv2
import numpy as np
from skimage.measure import shannon_entropy
from skimage.feature.texture import graycomatrix, graycoprops

def extract_features(region):
    features = []
    features.append(np.mean(region))
    features.append(np.std(region))
    features.append(np.var(region))
    features.append(shannon_entropy(region))
    edges = cv2.Canny(region, 50, 150)
    features.append(np.sum(edges > 0))
    try:
        glcm = graycomatrix(region, [1], [0], 256, symmetric=True, normed=True)
        features.append(graycoprops(glcm, 'contrast')[0, 0])
        features.append(graycoprops(glcm, 'homogeneity')[0, 0])
    except:
        features.append(0)
        features.append(0)
    return features

def detect_stroke_like_regions(image_path, window_size=64, stride=32, threshold=5.5):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError("Görüntü bulunamadı:", image_path)

    h, w = img.shape
    output_mask = np.zeros_like(img)

    for y in range(0, h - window_size + 1, stride):
        for x in range(0, w - window_size + 1, stride):
            patch = img[y:y+window_size, x:x+window_size]
            features = extract_features(patch)
            features_scaled = scaler.transform([features])
            dist, _ = knn.kneighbors(features_scaled)

            if dist[0][0] < threshold:
                output_mask[y:y+window_size, x:x+window_size] = 255

    return output_mask



In [16]:
def enhance_regions(image_path, mask):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    
    # Tüm görüntüye CLAHE uygula
    clahe_img = clahe.apply(img)
    
    # Orijinal görüntüyü kopyala
    enhanced = img.copy()
    
    # Sadece maske içindeki bölgeleri güncelle
    enhanced[mask == 255] = clahe_img[mask == 255]
    
    return enhanced


In [18]:
import matplotlib.pyplot as plt

def plot_distance_distribution(distances):
    plt.figure(figsize=(10, 6))
    plt.hist(distances, bins=50, color='skyblue', edgecolor='black')
    plt.xlabel("KNN Distance to Stroke Feature Vector")
    plt.ylabel("Patch Count")
    plt.title("Distribution of Patch Distances")
    plt.grid(True)
    plt.show()

test_img_path = "D:/ITU VCAMP/github/health-ai-project/data/all-rotated/10002.png"
mask = detect_stroke_like_regions(test_img_path)
distances = detect_stroke_like_regions(test_img_path)
cv2.imwrite("output_mask.png", mask)
cv2.imwrite("enhanced_image.png", enhanced_img)


True